In [29]:
from pathlib import Path
import pandas as pd

base = Path("../processed")

# carregar os arquivos limpos
geral = pd.read_csv(base / "geral_limpo.csv")
leads = pd.read_csv(base / "leads_limpo.csv")
visitas = pd.read_csv(base / "visitas_limpo.csv")
contratos = pd.read_csv(base / "contratos_limpo.csv")




In [ ]:
# ===============================================================
#  CRIAÇÃO DE MÉTRICAS E DF_FUNIL 
# ===============================================================


date_cols_funil = ["data_criacao", "data_venda", "data_perda"]
for col in date_cols_funil:
    # Garante que as colunas estão no formato datetime antes da subtração
    leads[col] = pd.to_datetime(leads[col], errors="coerce")


# -----------------------------
# Flags e Métricas
# -----------------------------
leads["flag_indicado"] = leads["indicado"].astype(int)
leads["flag_venda"] = leads["data_venda"].notna().astype(int)

# Métricas de ciclo (Dias) 
leads["dias_ate_venda"] = (leads["data_venda"] - leads["data_criacao"]).dt.days
leads["dias_ate_perda"] = (leads["data_perda"] - leads["data_criacao"]).dt.days


In [ ]:

base = Path("../processed")
geral = pd.read_csv(base / "geral_limpo.csv")


# linha de meta (primeira ocorrência de "Vendas")
linha_meta = geral[geral["metrica"] == "Vendas"].iloc[0]

# linha de realizado (segunda ocorrência de "Vendas")
linha_real = geral[geral["metrica"] == "Vendas"].iloc[1]

# ---------------------------
# 3. Selecionar período (Abr/24 → Mar/25)
# ---------------------------
colunas_periodo = [
    col for col in geral.columns 
    if "2024-" in col or "2025-" in col
]

# usar de Abril/24 até Março/25 (12 meses)
colunas_periodo = [c for c in colunas_periodo if "2024-04" <= c <= "2025-03"]

print("Meses usados no cálculo →", colunas_periodo)

# ---------------------------
# 4. Converter para série numérica
# ---------------------------
meta = linha_meta[colunas_periodo].astype(float)
realizado = linha_real[colunas_periodo].astype(float)

# ---------------------------
# 5. Cálculos
# ---------------------------
realizado_total = realizado.sum()
meta_total = meta.sum()
atingimento_medio = realizado_total / meta_total

# ---------------------------
# 6. Exibir diagnóstico
# ---------------------------
print(f"--- Diagnóstico: Atingimento Médio da Meta de Vendas ---")
print(f"Período: Abr/24 a Mar/25 ({len(colunas_periodo)} meses)")
print(f"Realizado Total: {realizado_total:,.0f} vendas")
print(f"Meta Total: {meta_total:,.0f} vendas")
print(f"Atingimento Médio da Meta: {atingimento_medio:.1%}")


Meses usados no cálculo → ['2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03']
--- Diagnóstico: Atingimento Médio da Meta de Vendas ---
Período: Abr/24 a Mar/25 (12 meses)
Realizado Total: 6,084 vendas
Meta Total: 7,285 vendas
Atingimento Médio da Meta: 83.5%


PERGUNTA 02: Qual coluna da base mostra se o lead foi indicado por um membro da Livance?



A coluna da base que mostra se o foi indicado por um membro da Livance é a coluna: "indicado" 

PERGUNTA 03: Qual canal de aquisição gerou o maior volume de leads?




In [34]:
# --- Análise 3: Taxa de Conversão Geral em Venda ---

# 1. Contar o Total de Leads (entrada do funil)
# Utiliza a contagem única de IDs no dataframe inteiro
total_leads = df_funil['lead_id'].nunique()

# 2. Contar o Total de Leads Vendidos
# Utiliza a flag_venda (onde 1 = VENDIDO) para contar leads únicos
total_vendas = df_funil[df_funil['flag_venda'] == 1]['lead_id'].nunique()

# 3. Calcular a Taxa de Conversão
# Fórmula: (Total de Vendas / Total de Leads)
taxa_conversao_geral = total_vendas / total_leads

# 4. Exibir o resultado formatado
print(f"--- Diagnóstico: Taxa de Conversão Geral ---")
print(f"Total de Leads (Volume de entrada): {total_leads:,.0f}")
print(f"Total de Vendas (Output final): {total_vendas:,.0f}")
print(f"Taxa de Conversão Geral: {taxa_conversao_geral:.2%}")

--- Diagnóstico: Taxa de Conversão Geral ---
Total de Leads (Volume de entrada): 52,098
Total de Vendas (Output final): 6,207
Taxa de Conversão Geral: 11.91%


PERGUNTA 04: Qual canal de aquisição gerou o maior volume de leads?

In [35]:
# --- Análise 4: Volume de Leads por Canal de Aquisição (utm_source) ---

# 1. Agrupar o df_funil pela coluna utm_source e contar os leads únicos
volume_por_canal = df_funil.groupby('utm_source')['lead_id'].nunique().reset_index()

# 2. Renomear a coluna de contagem para clareza
volume_por_canal.rename(columns={'lead_id': 'Total_Leads'}, inplace=True)

# 3. Ordenar o resultado para identificar o maior volume
volume_por_canal_ordenado = volume_por_canal.sort_values(
    by='Total_Leads', 
    ascending=False
)

# 4. Exibir o resultado e identificar o canal principal
print("--- Diagnóstico: Top Canais de Aquisição por Volume ---")
print(volume_por_canal_ordenado.head(5)) # Mostrando os 5 primeiros

# Identificando o canal com maior volume
canal_maior_volume = volume_por_canal_ordenado.iloc[0]

--- Diagnóstico: Top Canais de Aquisição por Volume ---
          utm_source  Total_Leads
8             google        19621
11         instagram        17539
33  whatsapp oficial         4574
4       desconhecido         3766
15     não informado         2066


In [ ]:
from IPython.display import display

# --- Análise 4: Ranking de Volume de Leads por Canal de Aquisição (utm_source) ---

# 1. Agrupar por 'utm_source' e contar o total de leads (IDs únicos)
df_volume_canal = df_funil.groupby('utm_source')['lead_id'].nunique().reset_index()

# 2. Renomear a coluna e ordenar pelo volume em ordem decrescente
df_volume_canal.columns = ['canal_aquisicao', 'total_leads']
df_volume_canal = df_volume_canal.sort_values(by='total_leads', ascending=False).reset_index(drop=True)

# 3. Calcular a participação de cada canal no volume total
total_geral = df_volume_canal['total_leads'].sum()
df_volume_canal['participacao_%'] = df_volume_canal['total_leads'] / total_geral

# 4. Formatar as colunas e exibir o resultado (Top 10 canais)
df_exibicao = df_volume_canal.head(10).copy()

# Aplica formatação de milhar com ponto e remove casas decimais
df_exibicao['total_leads'] = df_exibicao['total_leads'].apply(
    lambda x: f"{x:,.0f}".replace(",", "_").replace(".", ",").replace("_", ".")
)
# Aplica formatação de porcentagem com duas casas decimais
df_exibicao['participacao_%'] = df_exibicao['participacao_%'].apply(
    lambda x: f"{x:.2%}"
)

# Renomeia o index 
df_exibicao.index = df_volume_canal.index[:10]

print("--- Diagnóstico: Ranking de Volume de Leads por utm_source ---")
# Usei display para uma saída formatada e limpa no notebook
display(df_exibicao)

--- Diagnóstico: Ranking de Volume de Leads por utm_source ---


,canal_aquisicao,total_leads,participacao_%
0,google,19.621,37.66%
1,instagram,17.539,33.67%
2,whatsapp oficial,4.574,8.78%
3,desconhecido,3.766,7.23%
4,não informado,2.066,3.97%
5,typeform-blip,1.187,2.28%
6,typeform-ex membro,593,1.14%
7,typeform-indicação interna,500,0.96%
8,typeform-outros,446,0.86%
9,typeform-gentileza,425,0.82%


O canal de aquisição gerou o maior volume de leads foi o google com 37.66%